# Clustering Crypto

In [1]:
# Initial imports
import pandas as pd
import hvplot.pandas
from path import Path
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans


ModuleNotFoundError: No module named 'hvplot'

### Deliverable 1: Preprocessing the Data for PCA

In [ ]:
# Load the crypto_data.csv dataset.
file_path = "crypto_data.csv"
crypto_df = pd.read_csv(file_path)
crypto_df.head()



In [ ]:
# Keep all the cryptocurrencies that are being traded.
crypto_df = crypto_df.loc[(crypto_df["IsTrading"] == True)]
crypto_df

In [ ]:
# Remove the "IsTrading" column. 
crypto_df_traded = crypto_df.drop(columns={"IsTrading"})
crypto_df_traded

In [ ]:
# Remove rows that have at least 1 null value.
crypto_df_traded = crypto_df_traded.dropna()
crypto_df_traded

In [ ]:
# Keep the rows where coins are mined.
crypto_df_traded = crypto_df_traded.loc[(crypto_df_traded["TotalCoinsMined"] > 0)]



In [ ]:
# Create a new DataFrame that holds only the cryptocurrencies names.
crypto_df_names =pd.DataFrame(data=crypto_df_traded["Unnamed: 0"])
crypto_df_names


In [ ]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm.
crypto_df_traded = crypto_df_traded.set_index(["Unnamed: 0"])
crypto_df_traded = crypto_df_traded.drop(columns={"CoinName"})

In [ ]:
crypto_df_traded.head(10)

In [ ]:
# Use get_dummies() to create variables for text features.
X = crypto_df_traded.copy()
#X = X.drop("loan_status",axis=1)
X = pd.get_dummies(X, columns=["Algorithm","ProofType"])
X.head(10)

In [ ]:
# Standardize the data with StandardScaler().
from sklearn.preprocessing import StandardScaler
data_scaler = StandardScaler()
X_scaled = data_scaler.fit_transform(X)
print(X_scaled)

### Deliverable 2: Reducing Data Dimensions Using PCA

In [ ]:
# Using PCA to reduce dimension to three principal components.
from sklearn.decomposition import PCA
pca = PCA(n_components=3)

In [ ]:
# Create a DataFrame with the three principal components.
X_pca = pca.fit_transform(X_scaled)
pcs_df = pd.DataFrame(data = X_pca, columns=["PC1","PC2","PC3"])

In [ ]:
pcs_df = pcs_df.assign(CoinName='CoinName')

In [15]:
pcs_df

,PC1,PC2,PC3,CoinName
0,-0.333927,0.944216,-0.577450,CoinName
1,-0.317261,0.944359,-0.577966,CoinName
2,2.306129,1.776547,-0.649739,CoinName
3,-0.143708,-1.223464,0.209510,CoinName
4,-0.151550,-1.992294,0.377194,CoinName
...,...,...,...,...
527,2.470119,0.912165,-0.078763,CoinName
528,-0.331972,0.944112,-0.577483,CoinName
529,0.323815,-2.195273,0.432986,CoinName
530,-0.148089,-2.058034,0.455953,CoinName


In [16]:
pcs_df['CoinName'] = crypto_df_names["Unnamed: 0"].values
pcs_df = pcs_df.set_index('CoinName')
pcs_df.head(10)

,PC1,PC2,PC3
CoinName,,,
42,-0.333927,0.944216,-0.577450
404,-0.317261,0.944359,-0.577966
1337,2.306129,1.776547,-0.649739
BTC,-0.143708,-1.223464,0.209510
ETH,-0.151550,-1.992294,0.377194
LTC,-0.164290,-1.218110,-0.015559
DASH,-0.397454,1.307733,-0.543765
XMR,-0.145137,-2.265878,0.398448
ETC,-0.149992,-1.992384,0.377170


### Deliverable 3: Clustering Crytocurrencies Using K-Means

#### Finding the Best Value for `k` Using the Elbow Curve

In [ ]:
# Create an elbow curve to find the best value for K.
# YOUR CODE HERE


Running K-Means with `k=4`

In [ ]:
# Initialize the K-Means model.
# YOUR CODE HERE

# Fit the model
# YOUR CODE HERE

# Predict clusters
# YOUR CODE HERE

In [ ]:
# Create a new DataFrame including predicted clusters and cryptocurrencies features.
# Concatentate the crypto_df and pcs_df DataFrames on the same columns.
# YOUR CODE HERE

#  Add a new column, "CoinName" to the clustered_df DataFrame that holds the names of the cryptocurrencies. 
# YOUR CODE HERE

#  Add a new column, "Class" to the clustered_df DataFrame that holds the predictions.
# YOUR CODE HERE

# Print the shape of the clustered_df
print(clustered_df.shape)
clustered_df.head(10)

### Deliverable 4: Visualizing Cryptocurrencies Results

#### 3D-Scatter with Clusters

In [ ]:
# Creating a 3D-Scatter with the PCA data and the clusters
# YOUR CODE HERE


In [ ]:
# Create a table with tradable cryptocurrencies.
# YOUR CODE HERE

In [ ]:
# Print the total number of tradable cryptocurrencies.
# YOUR CODE HERE

In [ ]:
# Scaling data to create the scatter plot with tradable cryptocurrencies.
# YOUR CODE HERE

In [ ]:
# Create a new DataFrame that has the scaled data with the clustered_df DataFrame index.
# YOUR CODE HERE

# Add the "CoinName" column from the clustered_df DataFrame to the new DataFrame.
# YOUR CODE HERE

# Add the "Class" column from the clustered_df DataFrame to the new DataFrame. 
# YOUR CODE HERE

plot_df.head(10)

In [ ]:
# Create a hvplot.scatter plot using x="TotalCoinsMined" and y="TotalCoinSupply".
# YOUR CODE HERE
